In [1]:
print('OK')

OK


In [34]:
from langchain.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
# from langchain.output_parsers import JsonOutputParser
import json
from pydantic import BaseModel
from typing import List
from typing import List, Optional
import itertools
import requests

import pandas as pd
from pydantic import BaseModel, Field, field_validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback
from pydantic import BaseModel, Field
from typing import List, Optional

In [67]:


class Address(BaseModel):
    name: Optional[str] = Field(None, description="Name of the person or entity")
    address1: str = Field(..., description="Primary address line")
    address2: Optional[str] = Field(None, description="Secondary address line")
    address3: Optional[str] = Field(None, description="Tertiary address line")
    city: str = Field(..., description="City of the address")
    state: str = Field(..., description="State of the address")
    country: str = Field(..., description="Country of the address")
    country_code: Optional[str] = Field(None, description="Country code")
    zip: str = Field(..., description="ZIP or postal code")

class Contact(BaseModel):
    name: str = Field(..., description="Contact name")
    email: Optional[str] = Field(None, description="Contact email")
    contact_number: str = Field(..., description="Contact phone number")

class LineItem(BaseModel):
    line_no: str = Field(..., description="Line number")
    on_hand: str = Field(..., description="On-hand stock")
    to_buy: str = Field(..., description="To buy quantity")
    quantity: int = Field(..., description="Quantity of items")
    uom: str = Field(..., description="Unit of measure")
    unit_price: str = Field(..., description="Unit price of item")
    currency: str = Field(..., description="Currency used")
    part_numbers: str = Field(..., description="Part numbers")
    product_description: str = Field(..., description="Product description")
    spell_corrected_product_description: str = Field(..., description="Spell-corrected product description")

class Header(BaseModel):
    ship_to: Address = Field(..., description="Shipping address")
    bill_to: Address = Field(..., description="Billing address")
    vendor: Optional[Address] = Field(None, description="Vendor address")
    buyer_contact: Contact = Field(..., description="Buyer contact information")
    shipping_contact: Contact = Field(..., description="Shipping contact information")
    project_number: Optional[str] = Field(None, description="Project number")
    purchase_order_number: Optional[str] = Field(None, description="Purchase order number")
    job_name: Optional[str] = Field(None, description="Job name")
    job_number: str = Field(..., description="Job number")
    quote_number: Optional[str] = Field(None, description="Quote number")
    date_ordered: str = Field(..., description="Date the order was placed", format="date")
    delivery_date: str = Field(..., description="Date the delivery is expected", format="date")
    shipping_instructions: Optional[str] = Field(None, description="Shipping instructions")
    notes: Optional[str] = Field(None, description="Additional notes")
    ship_via: Optional[str] = Field(None, description="Shipping method")
    payment_terms: Optional[str] = Field(None, description="Payment terms")

class Extraction(BaseModel):
    header: Header = Field(..., description="Order header details")
    line_items: List[LineItem] = Field(..., description="List of line items in the order")

class Schema(BaseModel):
    extraction: List[Extraction] = Field(..., description="List of extraction details")


In [81]:
with open("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt", 'r') as f:
    text = f.read()

In [91]:
import os
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Load environment variables
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0)

# Define prompt for extraction
# prompt_template = ChatPromptTemplate.from_messages([
#     {"role": "system", "content": "Extract user information."},
#     {"role": "user", "content": "{text}"}
# ])

prompt_template = PromptTemplate(
    template="Extract all order details from the following text: {text}",
    input_variables=["text"]
)


# Create output parser using Pydantic model
# output_parser = PydanticOutputParser(pydantic_schema=Schema, input_type=str)
output_parser = PydanticOutputParser(pydantic_schema=Schema, pydantic_object=Schema(), input_type=str)


def extract_user_info(text: str) -> Schema:
    prompt = prompt_template.format_messages(text=text)
    response = llm(prompt)
    structured_output = output_parser.parse(response)
    return structured_output

# Read text file and extract information
# with open('user_data.txt', 'r') as file:
#     text_data = file.read()

user_info = extract_user_info(text)
print(user_info.json())


ValidationError: 1 validation error for Schema
extraction
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [68]:
from langchain.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import JsonOutputKeyToolsParser
import json

# Define a function to extract order details using function calling
@tool
def extract_order_details(text: str) -> dict:
    """Extracts order-related details from the provided text."""
    # In a real scenario, we would use some NLP models or API to extract this info
    # For now, we mock the extraction with a sample dictionary
    extracted_data = {
  "extraction": [
    {
      "header": {
        "ship_to": {
          "name": "John Doe",
          "address1": "123 Elm St",
          "address2": "Apt 4B",
          "address3": None,
          "city": "Springfield",
          "state": "IL",
          "country": "USA",
          "country_code": "US",
          "zip": "62704"
        },
        "bill_to": {
          "name": "John Doe",
          "address1": "456 Oak St",
          "address2": None,
          "address3": None,
          "city": "Springfield",
          "state": "IL",
          "country": "USA",
          "country_code": "US",
          "zip": "62704"
        },
        "vendor": {
          "name": "Tech Supplies Inc.",
          "address1": "789 Pine St",
          "address2": "Suite 300",
          "address3": None,
          "city": "Chicago",
          "state": "IL",
          "country": "USA",
          "country_code": "US",
          "zip": "60601"
        },
        "buyer_contact": {
          "name": "Alice Johnson",
          "email": "alice.johnson@example.com",
          "contact_number": "+1-555-123-4567"
        },
        "shipping_contact": {
          "name": "Bob Smith",
          "email": "bob.smith@example.com",
          "contact_number": "+1-555-765-4321"
        },
        "project_number": "PRJ-12345",
        "purchase_order_number": "PO-98765",
        "job_name": "New Construction",
        "job_number": "JOB-45678",
        "quote_number": "QUOTE-00123",
        "date_ordered": "2024-12-15",
        "delivery_date": "2024-12-25",
        "shipping_instructions": "Deliver between 9 AM and 5 PM.",
        "notes": "Handle with care.",
        "ship_via": "FedEx",
        "payment_terms": "Net 30"
      },
      "line_items": [
        {
          "line_no": "1",
          "on_hand": "20",
          "to_buy": "5",
          "quantity": 5,
          "uom": "pcs",
          "unit_price": "200.50",
          "currency": "USD",
          "part_numbers": "P-1234",
          "product_description": "Dell Laptop Model X123",
          "spell_corrected_product_description": "Dell Laptop Model X123"
        },
        {
          "line_no": "2",
          "on_hand": "50",
          "to_buy": "10",
          "quantity": 10,
          "uom": "pcs",
          "unit_price": "15.00",
          "currency": "USD",
          "part_numbers": "P-5678",
          "product_description": "Wireless Mouse Model M456",
          "spell_corrected_product_description": "Wireless Mouse Model M456"
        }
      ]
    }
  ]
}

    return extracted_data


In [69]:
extracted_data = {
  "extraction": [
    {
      "header": {
        "ship_to": {
          "name": "John Doe",
          "address1": "123 Elm St",
          "address2": "Apt 4B",
          "address3": None,
          "city": "Springfield",
          "state": "IL",
          "country": "USA",
          "country_code": "US",
          "zip": "62704"
        },
        "bill_to": {
          "name": "John Doe",
          "address1": "456 Oak St",
          "address2": None,
          "address3": None,
          "city": "Springfield",
          "state": "IL",
          "country": "USA",
          "country_code": "US",
          "zip": "62704"
        },
        "vendor": {
          "name": "Tech Supplies Inc.",
          "address1": "789 Pine St",
          "address2": "Suite 300",
          "address3": None,
          "city": "Chicago",
          "state": "IL",
          "country": "USA",
          "country_code": "US",
          "zip": "60601"
        },
        "buyer_contact": {
          "name": "Alice Johnson",
          "email": "alice.johnson@example.com",
          "contact_number": "+1-555-123-4567"
        },
        "shipping_contact": {
          "name": "Bob Smith",
          "email": "bob.smith@example.com",
          "contact_number": "+1-555-765-4321"
        },
        "project_number": "PRJ-12345",
        "purchase_order_number": "PO-98765",
        "job_name": "New Construction",
        "job_number": "JOB-45678",
        "quote_number": "QUOTE-00123",
        "date_ordered": "2024-12-15",
        "delivery_date": "2024-12-25",
        "shipping_instructions": "Deliver between 9 AM and 5 PM.",
        "notes": "Handle with care.",
        "ship_via": "FedEx",
        "payment_terms": "Net 30"
      },
      "line_items": [
        {
          "line_no": "1",
          "on_hand": "20",
          "to_buy": "5",
          "quantity": 5,
          "uom": "pcs",
          "unit_price": "200.50",
          "currency": "USD",
          "part_numbers": "P-1234",
          "product_description": "Dell Laptop Model X123",
          "spell_corrected_product_description": "Dell Laptop Model X123"
        },
        {
          "line_no": "2",
          "on_hand": "50",
          "to_buy": "10",
          "quantity": 10,
          "uom": "pcs",
          "unit_price": "15.00",
          "currency": "USD",
          "part_numbers": "P-5678",
          "product_description": "Wireless Mouse Model M456",
          "spell_corrected_product_description": "Wireless Mouse Model M456"
        }
      ]
    }
  ]
}


In [78]:
with open("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt", 'r') as f:
    text = f.read()

In [60]:
# # Set up the prompt template for LangChain
# prompt_template = PromptTemplate(
#     template="Extract all order details from the following text: {text}",
#     input_variables=["text"]
# )

# from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model="llama3.2:1b",
#     temperature=0)
# # # Initialize the LLM
# # llm = ChatOpenAI(
# #     model="gpt-4",  # or 'gpt-3.5-turbo'
# #     temperature=0.0,
# #     openai_api_key="your-openai-api-key"
# # )

# # JSON Output Parser
# class CustomJSONParser(JsonOutputKeyToolsParser):
#     def parse(self, text: str):
#         try:
#             return json.loads(text)
#         except json.JSONDecodeError:
#             raise ValueError("Invalid JSON format.")

# json_parser = CustomJSONParser()

# # Create the chain for structured output parsing
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt_template,
#     output_parser=json_parser
# )


In [61]:
from langchain.output_parsers import PydanticToolsParser

# Use PydanticToolsParser for structured parsing
parser = PydanticToolsParser(tools=[Schema])

In [72]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0)

structured_llm = llm.with_structured_output(Schema)

In [74]:
out = structured_llm.invoke(f"{text}")



In [75]:
print(out)

None


In [80]:
from pydantic import BaseModel, Field
from typing import List, Optional
import json
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser

# Define Pydantic Models
class ProductDetails(BaseModel):
    name: str = Field(..., description="Name of the product")
    quantity: int = Field(..., description="Quantity of the product")
    price: float = Field(..., description="Price of the product")

class BillingDetails(BaseModel):
    name: str = Field(..., description="Name of the billing contact")
    address: str = Field(..., description="Billing address")
    email: Optional[str] = Field(None, description="Email of the billing contact")

class ShippingDetails(BaseModel):
    name: str = Field(..., description="Name of the shipping contact")
    address: str = Field(..., description="Shipping address")
    tracking_id: Optional[str] = Field(None, description="Tracking ID for shipment")

class InvoiceDetails(BaseModel):
    invoice_number: str = Field(..., description="Invoice number")
    date: str = Field(..., description="Invoice date")
    billing_details: BillingDetails
    shipping_details: ShippingDetails
    products: List[ProductDetails]
    total_amount: float = Field(..., description="Total invoice amount")

# LangChain Setup
def parse_output_with_llm(raw_text: str) -> InvoiceDetails:
    # Create Pydantic Output Parser
    parser = PydanticOutputParser(pydantic_object=Schema)

    # Define Prompt Template
    template = PromptTemplate(
        template="Extract structured information from the text: {text}\n{format_instructions}",
        input_variables=["text"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    from langchain_ollama import ChatOllama

    llm = ChatOllama(
        model="llama3.2:1b",
        temperature=0)

    # Generate response
    prompt = template.format(text=raw_text)
    response = llm.invoke(prompt)

    # Parse LLM Output
    return parser.parse(response)

# Example Usage
def main():
    # Load text file
    # with open("sample_invoice.txt", "r") as file:
    #     raw_text = file.read()

    # Extract and parse data
    parsed_invoice = parse_output_with_llm(text)

    # Print formatted output
    print(parsed_invoice.json(indent=4))

if __name__ == "__main__":
    main()


ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=AIMessage(content='Here i..., 'total_tokens': 3687}), input_type=AIMessage]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

In [64]:
from langchain.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
# from langchain.output_parsers import JsonOutputParser
import json
from pydantic import BaseModel
from typing import List

# # Define the Pydantic models
# class Item(BaseModel):
#     product_id: str
#     product_name: str
#     quantity: int
#     price_per_unit: float

# class OrderDetails(BaseModel):
#     order_id: str
#     customer_name: str
#     shipping_address: str
#     status: str
#     delivery_date: str
#     total_amount: float
#     items: List[Item]

# Function to simulate extracting order details
# @tool
# def extract_order_details(text: str) -> dict:
#     """Extracts order-related details from the provided text."""
#     extracted_data = {
#         "order_id": "12345",
#         "customer_name": "John Doe",
#         "shipping_address": "123 Elm St, Springfield, IL",
#         "status": "Shipped",
#         "delivery_date": "2024-12-25",
#         "total_amount": 250.75,
#         "items": [
#             {
#                 "product_id": "A1",
#                 "product_name": "Laptop",
#                 "quantity": 1,
#                 "price_per_unit": 200.50
#             },
#             {
#                 "product_id": "B2",
#                 "product_name": "Mouse",
#                 "quantity": 1,
#                 "price_per_unit": 50.25
#             }
#         ]
#     }
#     return extracted_data


# Define the parser with tools
parser = PydanticToolsParser(tools=[Schema])

# Create the prompt template
prompt = PromptTemplate(
    template="""Extract the order details from the given text. Return the output in JSON format with the specified schema:
    {schema_description}

    Text:
    {text}
    """,
    input_variables=["schema_description", "text"]
)

from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0)

# Use LangChain's built-in JsonOutputParser for extracting JSON
# json_parser = JsonOutputParser()

# # Create the LangChain chain for structured output parsing
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt_template,
#     output_parser=parser
# )

# Format prompt with schema description
formatted_prompt = prompt.format(
    schema_description=Schema.model_json_schema(),  # Auto-generates schema description
    text=extracted_data
)

# Use the chain for structured parsing
chain = llm | parser  # Combine LLM and parser in a chain

# Run the chain with the formatted prompt
output = chain.invoke(formatted_prompt)



# Example order text
# order_text = """
#     Order ID: 12345
#     Customer Name: John Doe
#     Shipping Address: 123 Elm St, Springfield, IL
#     Status: Shipped
#     Delivery Date: 2024-12-25
#     Total Amount: $250.75
#     Items:
#     - Product ID: A1, Laptop, Quantity: 1, Price: $200.50
#     - Product ID: B2, Mouse, Quantity: 1, Price: $50.25
# """

# Run the chain to extract order details
# extracted_dat = chain.run({"text": text,"extracted_data": extracted_data})

# Parse the extracted data using Pydantic models
# try:
#     order = OrderDetails(**extracted_data)  # Pydantic validation
#     print(order.json(indent=2))  # Pretty-print the structured order data
# except ValueError as e:
#     print(f"Error parsing the extracted data: {e}")


In [65]:
print(output)

[]


In [47]:
extracted_dat

[]

In [48]:
# Extract JSON block using regex
import re
match = re.search(r'```json\n(.*?)\n```', extracted_dat, re.DOTALL)

if match:
    json_str = match.group(1)
    try:
        # Parse the JSON string
        data = json.loads(json_str)
        print(json.dumps(data, indent=4))  # Pretty print the extracted JSON
    except json.JSONDecodeError as e:
        print("Invalid JSON format:", e)
else:
    print("No JSON found in the text.")

TypeError: expected string or bytes-like object, got 'list'

In [44]:
parsed_data = parser.parse(data)

# Validate and print the output
try:
    order = Schema(**parsed_data)
    print(order.json(indent=2))
except ValueError as e:
    print(f"Error parsing the data: {e}")


NotImplementedError: 

In [11]:
text

'Extracted Text from Page:\n\n\nOCR Results for Image 1:\nROOFING WORK ORDER (Confidence: 0.9074893646359959)\nCrew: (Confidence: 0.9998794606765605)\nJob #: (Confidence: 0.9839771342886046)\n46500 (Confidence: 0.9973271356204866)\nStart Date: (Confidence: 0.8740533065162321)\nSupply Co: (Confidence: 0.9748557275163592)\nCrew Contact 8am (Confidence: 0.8319221732972323)\nNoon (Confidence: 0.990284264087677)\n4pm (Confidence: 0.9947997073543151)\nEmail Date:_ (Confidence: 0.7562557059771977)\n12 (Confidence: 0.9882900409558113)\n2} (Confidence: 0.3620489560004472)\nDelivery Communication (Confidence: 0.9997090073754178)\nDate to Deliver: (Confidence: 0.9975844441584428)\nRep: (Confidence: 0.9998525977134705)\nsent text (Confidence: 0.915932836799662)\nsent email (Confidence: 0.9634564005781879)\nAM (Confidence: 0.9994721936755578)\nPM (Confidence: 0.9994458982098452)\nPick Up Date (Confidence: 0.9640145454900017)\nAM (Confidence: 0.9999411635064507)\nPM (Confidence: 0.7201788671934961)\

In [20]:
from typing import List, Optional
import itertools
import requests

import pandas as pd
from pydantic import BaseModel, Field, field_validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback
from pydantic import BaseModel, Field
from typing import List, Optional

In [22]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Address(BaseModel):
    name: Optional[str] = Field(None, description="Name of the person or entity")
    address1: str = Field(..., description="Primary address line")
    address2: Optional[str] = Field(None, description="Secondary address line")
    address3: Optional[str] = Field(None, description="Tertiary address line")
    city: str = Field(..., description="City of the address")
    state: str = Field(..., description="State of the address")
    country: str = Field(..., description="Country of the address")
    country_code: Optional[str] = Field(None, description="Country code")
    zip: str = Field(..., description="ZIP or postal code")

class Contact(BaseModel):
    name: str = Field(..., description="Contact name")
    email: Optional[str] = Field(None, description="Contact email")
    contact_number: str = Field(..., description="Contact phone number")

class LineItem(BaseModel):
    line_no: str = Field(..., description="Line number")
    on_hand: str = Field(..., description="On-hand stock")
    to_buy: str = Field(..., description="To buy quantity")
    quantity: int = Field(..., description="Quantity of items")
    uom: str = Field(..., description="Unit of measure")
    unit_price: str = Field(..., description="Unit price of item")
    currency: str = Field(..., description="Currency used")
    part_numbers: str = Field(..., description="Part numbers")
    product_description: str = Field(..., description="Product description")
    spell_corrected_product_description: str = Field(..., description="Spell-corrected product description")

class Header(BaseModel):
    ship_to: Address = Field(..., description="Shipping address")
    bill_to: Address = Field(..., description="Billing address")
    vendor: Optional[Address] = Field(None, description="Vendor address")
    buyer_contact: Contact = Field(..., description="Buyer contact information")
    shipping_contact: Contact = Field(..., description="Shipping contact information")
    project_number: Optional[str] = Field(None, description="Project number")
    purchase_order_number: Optional[str] = Field(None, description="Purchase order number")
    job_name: Optional[str] = Field(None, description="Job name")
    job_number: str = Field(..., description="Job number")
    quote_number: Optional[str] = Field(None, description="Quote number")
    date_ordered: str = Field(..., description="Date the order was placed", format="date")
    delivery_date: str = Field(..., description="Date the delivery is expected", format="date")
    shipping_instructions: Optional[str] = Field(None, description="Shipping instructions")
    notes: Optional[str] = Field(None, description="Additional notes")
    ship_via: Optional[str] = Field(None, description="Shipping method")
    payment_terms: Optional[str] = Field(None, description="Payment terms")

class Extraction(BaseModel):
    header: Header = Field(..., description="Order header details")
    line_items: List[LineItem] = Field(..., description="List of line items in the order")

class Schema(BaseModel):
    extraction: List[Extraction] = Field(..., description="List of extraction details")


In [23]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0)

In [26]:
with open("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt", 'r') as f:
    text = f.read()

In [28]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


chain = llm | PydanticToolsParser(tools=[Schema])


In [29]:
chain.invoke(text)

[]

In [86]:
!pip install langchain_openai

  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/ed/54/63c8264d7dbc3bf31ba61bf97740fdd76386b2d4f9a58f58afd3961ce7d7/langchain_openai-0.2.14-py3-none-any.whl.metadata
  Obtaining dependency information for openai<2.0.0,>=1.58.1 from https://files.pythonhosted.org/packages/8e/5a/d22cd07f1a99b9e8b3c92ee0c1959188db4318828a3d88c9daac120bdd69/openai-1.58.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
   -------- ------------------------------- 10.2/50.9 kB ? eta -:--:--
   ---------------------------------------- 50.9/50.9 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/454.3 kB ? eta -:--:--
   --------------- ------------------------ 174.1/454.3 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 454.3/454.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Suc


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.2:1b",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

The 2020 World Series between the Los Angeles Dodgers and the Tampa Bay Rays was played at Globe Life Field in Arlington, Texas.


In [15]:
from openai import OpenAI

# client = OpenAI()

tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string"}
              },
          },
      },
  }
]

completion = client.chat.completions.create(
  model="llama3.2:1b",
  messages=[{"role": "user", "content": "What's the weather like in Paris today?"}],
  tools=tools,
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_xfe98aqv', function=Function(arguments='{"location":"Paris"}', name='get_weather'), type='function', index=0)]


In [16]:
tools = [
    {
      "type": "function",
      "function": {
        "name": "extract_order_data",
        "description": "Extracts structured data including headers and line items for orders based on the provided schema.",
        "parameters": {
          "type": "object",
          "properties": {
            "schema": {
              "type": "object",
              "description": "JSON schema defining the structure of the order data.",
              "properties": {
                "extraction": {
                  "type": "array",
                  "items": {
                    "type": "object",
                    "properties": {
                      "header": {
                        "type": "object",
                        "properties": {
                          "ship_to": {
                            "type": "object",
                            "properties": {
                              "name": { "type": "string" },
                              "address1": { "type": "string" },
                              "address2": { "type": "string" },
                              "address3": { "type": "string" },
                              "city": { "type": "string" },
                              "state": { "type": "string" },
                              "country": { "type": "string" },
                              "country_code": { "type": "string" },
                              "zip": { "type": "string" }
                            },
                            "required": ["address1", "state", "zip"]
                          },
                          "bill_to": {
                            "type": "object",
                            "properties": {
                              "name": { "type": "string" },
                              "address1": { "type": "string" },
                              "address2": { "type": "string" },
                              "address3": { "type": "string" },
                              "city": { "type": "string" },
                              "state": { "type": "string" },
                              "country": { "type": "string" },
                              "country_code": { "type": "string" },
                              "zip": { "type": "string" }
                            },
                            "required": ["name", "address1", "city", "state", "zip"]
                          },
                          "vendor": {
                            "type": "object",
                            "properties": {
                              "name": { "type": "string" },
                              "address1": { "type": "string" },
                              "address2": { "type": "string" },
                              "address3": { "type": "string" },
                              "city": { "type": "string" },
                              "state": { "type": "string" },
                              "country": { "type": "string" },
                              "country_code": { "type": "string" },
                              "zip": { "type": "string" }
                            }
                          },
                          "buyer_contact": {
                            "type": "object",
                            "properties": {
                              "name": { "type": "string" },
                              "email": { "type": "string" },
                              "contact_number": { "type": "string" }
                            },
                            "required": ["name", "contact_number"]
                          },
                          "shipping_contact": {
                            "type": "object",
                            "properties": {
                              "name": { "type": "string" },
                              "email": { "type": "string" },
                              "contact_number": { "type": "string" }
                            },
                            "required": ["name", "contact_number"]
                          },
                          "project_number": { "type": "string" },
                          "purchase_order_number": { "type": "string" },
                          "job_name": { "type": "string" },
                          "job_number": { "type": "string" },
                          "quote_number": { "type": "string" },
                          "date_ordered": { "type": "string", "format": "date" },
                          "delivery_date": { "type": "string", "format": "date" },
                          "shipping_instructions": { "type": "string" },
                          "notes": { "type": "string" },
                          "ship_via": { "type": "string" },
                          "payment_terms": { "type": "string" }
                        },
                        "required": ["ship_to", "bill_to", "buyer_contact", "shipping_contact", "job_number", "date_ordered", "delivery_date", "shipping_instructions"]
                      },
                      "line_items": {
                        "type": "array",
                        "items": {
                          "type": "object",
                          "properties": {
                            "line_no": { "type": "string" },
                            "on_hand": { "type": "string" },
                            "to_buy": { "type": "string" },
                            "quantity": { "type": "integer" },
                            "uom": { "type": "string" },
                            "unit_price": { "type": "string" },
                            "currency": { "type": "string" },
                            "part_numbers": { "type": "string" },
                            "product_description": { "type": "string" },
                            "spell_corrected_product_description": { "type": "string" }
                          },
                          "required": ["line_no", "quantity", "part_numbers", "product_description", "spell_corrected_product_description"]
                        }
                      }
                    },
                    "required": ["header", "line_items"]
                  }
                }
              },
              "required": ["extraction"]
            }
          },
          "required": ["schema"]
        }
      }
    }
  ]


In [17]:
with open("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt", 'r') as f:
    text = f.read()

In [18]:
completion = client.chat.completions.create(
  model="llama3.2:1b",
  messages=[{"role": "user", "content": f"You have to extract all given information from the given text - {text}"}],
  tools=tools,
)

print(completion.choices[0].message.tool_calls)

None


In [2]:
from typing import List, Optional
import itertools
import requests

import pandas as pd
from pydantic import BaseModel, Field, field_validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback


In [3]:


class Address(BaseModel):
    name: Optional[str] = Field(None, description="Name of the person or entity")
    address1: str = Field(..., description="Primary address line")
    address2: Optional[str] = Field(None, description="Secondary address line")
    address3: Optional[str] = Field(None, description="Tertiary address line")
    city: str = Field(..., description="City of the address")
    state: str = Field(..., description="State of the address")
    country: str = Field(..., description="Country of the address")
    country_code: Optional[str] = Field(None, description="Country code")
    zip: str = Field(..., description="ZIP or postal code")

class Contact(BaseModel):
    name: str = Field(..., description="Contact name")
    email: Optional[str] = Field(None, description="Contact email")
    contact_number: str = Field(..., description="Contact phone number")

class LineItem(BaseModel):
    line_no: str = Field(..., description="Line number")
    on_hand: str = Field(..., description="On-hand stock")
    to_buy: str = Field(..., description="To buy quantity")
    quantity: int = Field(..., description="Quantity of items")
    uom: str = Field(..., description="Unit of measure")
    unit_price: str = Field(..., description="Unit price of item")
    currency: str = Field(..., description="Currency used")
    part_numbers: str = Field(..., description="Part numbers")
    product_description: str = Field(..., description="Product description")
    spell_corrected_product_description: str = Field(..., description="Spell-corrected product description")

class Header(BaseModel):
    ship_to: Address = Field(..., description="Shipping address")
    bill_to: Address = Field(..., description="Billing address")
    vendor: Optional[Address] = Field(None, description="Vendor address")
    buyer_contact: Contact = Field(..., description="Buyer contact information")
    shipping_contact: Contact = Field(..., description="Shipping contact information")
    project_number: Optional[str] = Field(None, description="Project number")
    purchase_order_number: Optional[str] = Field(None, description="Purchase order number")
    job_name: Optional[str] = Field(None, description="Job name")
    job_number: str = Field(..., description="Job number")
    quote_number: Optional[str] = Field(None, description="Quote number")
    date_ordered: str = Field(..., description="Date the order was placed", format="date")
    delivery_date: str = Field(..., description="Date the delivery is expected", format="date")
    shipping_instructions: Optional[str] = Field(None, description="Shipping instructions")
    notes: Optional[str] = Field(None, description="Additional notes")
    ship_via: Optional[str] = Field(None, description="Shipping method")
    payment_terms: Optional[str] = Field(None, description="Payment terms")

class Extraction(BaseModel):
    header: Header = Field(..., description="Order header details")
    line_items: List[LineItem] = Field(..., description="List of line items in the order")

class Schema(BaseModel):
    extraction: List[Extraction] = Field(..., description="List of extraction details")


In [4]:

# Create schema and extraction validator using from_pydantic
schema, extraction_validator = from_pydantic(
    Schema,
    description="Extract details for orders including ship-to, bill-to, contact info, line items, and order details.",
    examples=[
        (
            "Example order with details",
            {
                "extraction": [
                    {
                        "header": {
                            "ship_to": {
                                "address1": "123 Main St",
                                "city": "Anytown",
                                "state": "NY",
                                "zip": "10001",
                                "country": "USA"
                            },
                            "bill_to": {
                                "name": "John Doe",
                                "address1": "456 Elm St",
                                "city": "Othertown",
                                "state": "CA",
                                "zip": "20002",
                                "country": "USA"
                            },
                            "buyer_contact": {
                                "name": "Jane Buyer",
                                "contact_number": "123-456-7890"
                            },
                            "shipping_contact": {
                                "name": "John Shipping",
                                "contact_number": "987-654-3210"
                            },
                            "job_number": "J12345",
                            "date_ordered": "2024-12-22",
                            "delivery_date": "2025-01-15",
                            "shipping_instructions": "Handle with care",
                            "notes": "Rush delivery"
                        },
                        "line_items": [
                            {
                                "line_no": "1",
                                "quantity": 10,
                                "part_numbers": "ABC123",
                                "product_description": "Item A",
                                "spell_corrected_product_description": "Item A"
                            }
                        ]
                    }
                ]
            }
        )
    ],
    many=True,
)


In [ ]:
import os
import fitz  # PyMuPDF
import easyocr

def process_pdfs(input_folder, output_folder):
    # Ensure output folders for images and text exist
    image_output_folder = os.path.join(output_folder, "images")
    text_output_folder = os.path.join(output_folder, "text")
    os.makedirs(image_output_folder, exist_ok=True)
    os.makedirs(text_output_folder, exist_ok=True)
    
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])

    # Iterate through all PDF files in the input folder
    for pdf_file in os.listdir(input_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(input_folder, pdf_file)
            print(f"Processing PDF: {pdf_file}")

            # Open the PDF
            doc = fitz.open(pdf_path)

            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                
                # Create a single text file per page to combine all outputs
                text_filename = os.path.join(
                    text_output_folder, f"{os.path.splitext(pdf_file)[0]}_page_{page_num + 1}.txt"
                )
                with open(text_filename, "w", encoding="utf-8") as text_file:
                    # Extract page text
                    text = page.get_text("text")
                    text_file.write("Extracted Text from Page:\n")
                    text_file.write(text + "\n\n")
                    print(f"Saved page text for Page {page_num + 1}: {text_filename}")

                    # Extract images and perform OCR
                    images = page.get_images(full=True)
                    for img_index, img in enumerate(images):
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_bytes = base_image["image"]
                        image_filename = os.path.join(
                            image_output_folder, 
                            f"{os.path.splitext(pdf_file)[0]}_page_{page_num + 1}_img_{img_index + 1}.png"
                        )
                        with open(image_filename, "wb") as img_file:
                            img_file.write(image_bytes)
                        print(f"Saved image: {image_filename}")

                        # Perform OCR on the image
                        ocr_results = reader.readtext(image_filename)
                        text_file.write(f"OCR Results for Image {img_index + 1}:\n")
                        for (_, ocr_text, prob) in ocr_results:
                            text_file.write(f"{ocr_text} (Confidence: {prob})\n")
                        text_file.write("\n")
                        print(f"Saved OCR results for image: {image_filename}")

            # Close the document
            doc.close()

    print("Processing complete.")

# Example usage
input_folder = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data'
output_folder = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output'
process_pdfs(input_folder, output_folder)


In [5]:
with open("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt", 'r') as f:
    conversation = f.read()

In [6]:
from kor.documents.html import MarkdownifyHTMLProcessor

doc = Document(page_content=conversation)


md = MarkdownifyHTMLProcessor().process(doc)

split_docs = RecursiveCharacterTextSplitter().split_documents([md])

In [7]:
split_docs

[Document(metadata={}, page_content='Extracted Text from Page:\nOCR Results for Image 1:\nROOFING WORK ORDER (Confidence: 0.9074893646359959)\nCrew: (Confidence: 0.9998794606765605)\nJob #: (Confidence: 0.9839771342886046)\n46500 (Confidence: 0.9973271356204866)\nStart Date: (Confidence: 0.8740533065162321)\nSupply Co: (Confidence: 0.9748557275163592)\nCrew Contact 8am (Confidence: 0.8319221732972323)\nNoon (Confidence: 0.990284264087677)\n4pm (Confidence: 0.9947997073543151)\nEmail Date:\\_ (Confidence: 0.7562557059771977)\n12 (Confidence: 0.9882900409558113)\n2} (Confidence: 0.3620489560004472)\nDelivery Communication (Confidence: 0.9997090073754178)\nDate to Deliver: (Confidence: 0.9975844441584428)\nRep: (Confidence: 0.9998525977134705)\nsent text (Confidence: 0.915932836799662)\nsent email (Confidence: 0.9634564005781879)\nAM (Confidence: 0.9994721936755578)\nPM (Confidence: 0.9994458982098452)\nPick Up Date (Confidence: 0.9640145454900017)\nAM (Confidence: 0.9999411635064507)\nPM

In [8]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma
# # from langchain.embeddings import OpenAIEmbeddings
# # from langchain.llms import OpenAI
# from langchain.document_loaders import DirectoryLoader
# from langchain.document_loaders import TextLoader
# from langchain_community import embeddings


# loader = DirectoryLoader("../output/text/", glob = "./*.txt", loader_cls= TextLoader)

# document = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
# split_docs = text_splitter.split_documents(document)


# persist_directory = 'db'

# # 2. Convert documents to Embeddings and store them
# vectorstore = Chroma.from_documents(
#     documents=text,
#     collection_name="rag-chroma",
#     embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
# )

# retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [9]:
print(split_docs[0])

page_content='Extracted Text from Page:
OCR Results for Image 1:
ROOFING WORK ORDER (Confidence: 0.9074893646359959)
Crew: (Confidence: 0.9998794606765605)
Job #: (Confidence: 0.9839771342886046)
46500 (Confidence: 0.9973271356204866)
Start Date: (Confidence: 0.8740533065162321)
Supply Co: (Confidence: 0.9748557275163592)
Crew Contact 8am (Confidence: 0.8319221732972323)
Noon (Confidence: 0.990284264087677)
4pm (Confidence: 0.9947997073543151)
Email Date:\_ (Confidence: 0.7562557059771977)
12 (Confidence: 0.9882900409558113)
2} (Confidence: 0.3620489560004472)
Delivery Communication (Confidence: 0.9997090073754178)
Date to Deliver: (Confidence: 0.9975844441584428)
Rep: (Confidence: 0.9998525977134705)
sent text (Confidence: 0.915932836799662)
sent email (Confidence: 0.9634564005781879)
AM (Confidence: 0.9994721936755578)
PM (Confidence: 0.9994458982098452)
Pick Up Date (Confidence: 0.9640145454900017)
AM (Confidence: 0.9999411635064507)
PM (Confidence: 0.7201788671934961)
CONTRACTING (

In [10]:
# from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model='llama3.2:1b',temperature=0)


from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0)


In [11]:
# llm = ChatOpenAI(temperature=0, model='gpt-4o')


In [12]:
chain = create_extraction_chain(
    llm,
    schema,
    encoder_or_encoder_class="csv",
    validator=extraction_validator,
    input_formatter="triple_quotes",
)

NotImplementedError: CSV Encoder does not yet support embedded lists or objects (attribute `extraction`).

In [ ]:
with get_openai_callback() as cb:
    document_extraction_results = await extract_from_documents(
        chain, split_docs, max_concurrency=2, use_uid=False, return_exceptions=True
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [13]:
from tqdm.asyncio import tqdm
from tqdm import tqdm as sync_tqdm

# Assuming split_docs is a list of documents you're processing
async def extract_with_progress(chain, split_docs):
    document_extraction_results = []
    
    # Use tqdm with async iteration for progress tracking
    for doc in tqdm(split_docs, desc="Extracting documents", dynamic_ncols=True):
        result = await extract_from_documents(
            chain, [doc], max_concurrency=2, use_uid=False, return_exceptions=True
        )
        document_extraction_results.append(result)
        break
    
    return document_extraction_results

# Usage in your async function
with get_openai_callback() as cb:
    # Use the sync version of tqdm here as we're not iterating async directly
    document_extraction_results = await extract_with_progress(chain, split_docs)
    
    # After extraction, print stats
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")


Extracting documents:   0%|          | 0/2 [03:08<?, ?it/s]

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [14]:
document_extraction_results

[[{'uid': '0',
   'source_uid': '0',
   'data': {},
   'raw': '{\n    "customerInformation": {\n        "Derek": {\n            "name": "Derek",\n            "address": "Crown Point Rd, Man, (701) 390"\n        },\n        "Man": {\n            "name": "Man",\n            "address": "(701) 390"\n        }\n    },\n    "crewInformation": {\n        "Mefcl": {\n            "phone": "2lss",\n            "email": "Seles-2lss"\n        },\n        "Seles-2lss": {\n            "phone": "2lss",\n            "email": "Seles-2lss"\n        }\n    },\n    "productionInformation": {\n        "Crystal": {\n            "phone": "Mefcl",\n            "email": "Seles-2lss"\n        },\n        "Build Communication": {\n            "phone": "Mefcl",\n            "email": "Seles-2lss"\n        }\n    },\n    "deliveryInstructions": {\n        "placeMaterialsOnTheLeftSideOfTheDrivewayInTheFrontOfTheGarage": [\n            "Place materials on the LEFT SIDE OF THE DRIVEWAY, IN FRONT OF THE GARAGE."\n     

In [15]:
validated_data = list(
    itertools.chain.from_iterable(
        extraction["validated_data"] for extraction in document_extraction_results
    )
)

TypeError: list indices must be integers or slices, not str

In [ ]:
len(validated_data)


In [ ]:
pd.DataFrame(record.dict() for record in validated_data)


Testing OLLAMA

In [1]:
!pip install ollama


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install --upgrade llama

  Using cached llama-0.1.1.tar.gz (387 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "C:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        Fil

In [16]:
!pip install py-magic httpx PyPDF2
!pip install ollama==0.4.3

ERROR: Could not find a version that satisfies the requirement py-magic (from versions: none)
ERROR: No matching distribution found for py-magic

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
!pip install pdfplumber

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/d0/87/415cb472981a8d2e36beeeadf074ebb686cc2bfe8d18de973232da291bd5/pdfplumber-0.11.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 165.2 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 225.2 kB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six==20231228 from https://files.pythonhosted.org/packages/eb/9c/e46fe7502b32d7db6af6e36a9105abb93301fa1ec475b5ddcba8b35ae23a/pdfminer.six-20231228-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/25/bd/56d9ec6b9f0fc4e0d95288759f3179f0fcd34b1a1526b75673d2f6d5196f/pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata
     ---------------------------------------- 0.0


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install pdf2image

  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for pillow from https://files.pythonhosted.org/packages/dc/83/1470c220a4ff06cd75fc609068f6605e567ea51df70557555c2ab6516b2c/pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
    --------------------------------------- 0.1/2.6 MB 1.1 MB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.6 MB 2.8 MB/s eta 0:00:01
   --------- ------------------------------ 0.6/2.6 MB 4.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.4/2.6 MB 8.0 MB/s eta 0:00:01
   ----------------------- ---------------- 1.5/2.6 MB 8.2 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.6 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------  2.6/2.6 MB 8.6 MB/s 


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
!pip install pytesseract


  Obtaining dependency information for pytesseract from https://files.pythonhosted.org/packages/7a/33/8312d7ce74670c9d39a532b2c246a853861120486be9443eebf048043637/pytesseract-0.3.13-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
!pip install pdf2text


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
     - -------------------------------------- 0.2/4.2 MB 3.3 MB/s eta 0:00:02
     ----- ---------------------------------- 0.5/4.2 MB 5.5 MB/s eta 0:00:01
     ---------- ----------------------------- 1.1/4.2 MB 8.8 MB/s eta 0:00:01
     --------------- ------------------------ 1.6/4.2 MB 9.3 MB/s eta 0:00:01
     --------------- ------------------------ 1.6/4.2 MB 9.3 MB/s eta 0:00:01
     ---------------------- ----------------- 2.4/4.2 MB 9.0 MB/s eta 0:00:01
     ------------------------------ --------- 3.2/4.2 MB 10.2 MB/s eta 0:00:01
     ------------------------------ --------- 3.2/4.


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
!pip install pymupdf

  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/46/72/8c5bbf817aacebe21a454f3ade8ee4b5b17afe698bb73d65c4ca23a89a87/pymupdf-1.25.1-cp39-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/16.6 MB 825.8 kB/s eta 0:00:20
    --------------------------------------- 0.3/16.6 MB 2.3 MB/s eta 0:00:08
   - -------------------------------------- 0.7/16.6 MB 4.2 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/16.6 MB 5.0 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/16.6 MB 6.0 MB/s eta 0:00:03
   ---- ----------------------------------- 2.0/16.6 MB 6.7 MB/s eta 0:00:03
   ----- ---------------------------------- 2.3/16.6 MB 7.0 MB/s eta 0:00:03
   ------- -------------------------------- 3.2/16.6 MB 8.1 MB/s eta 0:00:02
   --------- ----------------------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
!pip install pytesseract opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.8 MB 2.4 MB/s eta 0:00:17
    --------------------------------------- 0.5/38.8 MB 4.5 MB/s eta 0:00:09
   - -------------------------------------- 1.1/38.8 MB 7.1 MB/s eta 0:00:06
   - -------------------------------------- 1.5/38.8 MB 8.6 MB/s eta 0:00:05
   - -------------------------------------- 1.5/38.8 MB 6.1 MB/s eta 0:00:07
   -- ------------------------------------- 2.6/38.8 MB 9.1 MB/s eta 0:00:04
   --- ------------------------------------ 3.1/38.8 MB 9.4 MB/s eta 0:00:04
   --- ------------------------------------ 3.1/38.8 MB 8.2 MB/s eta 0:00:05
   ---- --------------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import fitz  # pymupdf

def extract_table_and_images(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)

    # Extracting tables (if any) from each page
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        
        # Extract text and look for table structures
        text = page.get_text("text")
        print(text)
        
        # You can implement custom logic to recognize tables in the text here
        # For now, just print the text
        print(f"Page {page_num + 1} Text:\n{text[:500]}")  # Printing a snippet of text
        
        # Extract images
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_page_{page_num + 1}_img_{img_index + 1}.png"
            
            # Save the image
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
            print(f"Saved image: {image_filename}")

    # Close the document
    doc.close()

# Example usage
pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'
print(extract_table_and_images(pdf_path))



Page 1 Text:

Saved image: image_page_1_img_1.png
None


In [41]:
import cv2
import pytesseract

def extract_text_from_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use Pytesseract to do OCR on the image
    text = pytesseract.image_to_string(gray)

    # Print the extracted text
    print("Extracted Text:\n", text)

# Example usage
image_path = "C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/research/image_page_1_img_1.png"
extract_text_from_image(image_path)


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [48]:
!pip install google-cloud-vision==3.1.1 


  Obtaining dependency information for google-cloud-vision==3.1.1 from https://files.pythonhosted.org/packages/be/ea/8d2780031c00c60d53f285f2a684fe2964ee6655f5e145a676bd14e68f70/google_cloud_vision-3.1.1-py2.py3-none-any.whl.metadata
  Using cached google_cloud_vision-3.1.1-py2.py3-none-any.whl.metadata (6.1 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for grpcio-status<2.0.dev0,>=1.33.2 from https://files.pythonhosted.org/packages/40/ba/dc535631a9dffa421b327ebfc961911af54c396aa5324efd122a94f72464/grpcio_status-1.68.0-py3-none-any.whl.metadata
  Using cached grpcio_status-1.68.0-py3-none-any.whl.metadata (1.1 kB)
  Obtaining dependency information for grpcio-status<2.0.dev0,>=1.33.2 from https://files.pythonhosted.org/packages/05/18/56999a1da3577d8ccc8698a575d6638e15fe25650cc88b2ce0a087f180b9/grpcio_status-1.67.1-py3-none-any.whl.metadata
  Usi


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
from google.cloud.vision import ImageAnnotatorClient
from google.cloud.vision_v1 import types  # Use this if `types` import still fails

def extract_text_from_image(image_path):
    # Initialize the client
    client = ImageAnnotatorClient()

    # Read the image content
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    # Perform text detection
    image = types.Image(content=content)  # Ensure you're using the correct `types` import
    response = client.text_detection(image=image)
    texts = response.text_annotations

    # Print recognized text
    for text in texts:
        print('Text:', text.description)

image_path = "C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/research/image_page_1_img_1.png"
extract_text_from_image(image_path)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data", glob="**/*.pdf")
books = loader.load()
len(books)

Error loading file C:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\data\46500152___20240912_060559-redacted.pdf


ImportError: failed to find libmagic.  Check your installation

In [44]:
!pip install tesserocr

     ---------------------------------------- 0.0/71.8 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/71.8 kB ? eta -:--:--
     ---------------- --------------------- 30.7/71.8 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 71.8/71.8 kB 492.6 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      Failed to extract tesseract version from executable: [WinError 2] The system cannot find the file specified
      Supporting tesseract v3.04.00
      Tesseract major version 3
      Traceback (most recent call last):
        File "<string>", line 239, in get_build_args
        File "<string>", line 105, in package_config
        File "C:\Users\kisha\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1026, in __init__
          self._execute_child(args, executable, preexec_fn, close_fds,
        File "C:\Users\kisha\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1538, in _execute_child
          hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      FileNotFoundError: [WinError 2] The system cannot find the file specified
 

In [1]:
!pip install pdfplumber


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pdfplumber

# Open the PDF file
with pdfplumber.open('C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf') as pdf:
    text = ""
    
    # Loop through each page and extract text
    for page in pdf.pages:
        text += page.extract_text() + "\n"

# Print the extracted text
print(text)


ImportError: cannot import name 'PDFStackT' from 'pdfminer.pdfinterp' (c:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\venv\Lib\site-packages\pdfminer\pdfinterp.py)

In [ ]:
import tesserocr
from PIL import Image

def extract_text_from_image(image_path):
    # Open the image using PIL
    image = Image.open(image_path)

    # Perform OCR
    text = tesserocr.image_to_text(image)

    # Print the extracted text
    print("Extracted Text:\n", text)

# Example usage
image_path = "path/to/your/image.png"
extract_text_from_image(image_path)


ModuleNotFoundError: No module named 'tesserocr'

In [34]:
from pdf2text import Converter

def extract_text_from_pdf(pdf_path):
    # Initialize the Converter
    with Converter(pdf_path) as conv:
        # Extract text
        text = conv.get_text()
    return text

pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)


ImportError: cannot import name 'Extractor' from partially initialized module 'pdf2text' (most likely due to a circular import) (c:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\venv\Lib\site-packages\pdf2text\__init__.py)

In [29]:
import pytesseract
from pdf2image import convert_from_path

# Function to extract text from a PDF file using OCR
def extract_text_from_pdf_with_ocr(pdf_path):
    # Convert PDF pages to images
    pages = convert_from_path(pdf_path, 300)  # Adjust DPI if necessary

    text = ""
    for page in pages:
        # Extract text using pytesseract
        text += pytesseract.image_to_string(page) + "\n"
    
    return text

# Example usage
pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'
extracted_text = extract_text_from_pdf_with_ocr(pdf_path)
print(extracted_text)


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [32]:
import pdfplumber

# Function to extract text from a PDF file with detailed content
def extract_text_from_pdf_with_pdfplumber(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Example usage
pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'
extracted_text = extract_text_from_pdf_with_pdfplumber(pdf_path)
print(extracted_text)


In [31]:
import PyPDF2

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

# Example usage
pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)


In [20]:
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
from PIL import Image
import base64
import io

# Path to the PDF file
pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'

# Extract text from PDF
reader = PdfReader(pdf_path)
text_content = ''
for page in reader.pages:
    text_content += page.extract_text()

# Extract images from PDF
images = convert_from_path(pdf_path)

# Preprocess images: Convert images to Base64 for easier handling
def preprocess_image(image):
    with io.BytesIO() as buffer:
        image.save(buffer, format="JPEG")
        img_str = base64.b64encode(buffer.getvalue()).decode()
    return img_str

image_strs = [preprocess_image(img) for img in images]

# Format data for LLM
data = {
    'text': text_content,
    'images': image_strs  # or include them as attachments if supported
}

# Send to LLM (pseudo code, replace with actual API calls)
# response = llm_api.send(data)


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [12]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data", glob="**/*.pdf")
books = loader.load()
len(books)

Error loading file C:\Users\kisha\OneDrive\Desktop\revalgo assignment\revalgo-assignment\data\46500152___20240912_060559-redacted.pdf


ImportError: failed to find libmagic.  Check your installation

In [13]:
# !pip install langchain

In [17]:
# from langchain.llms import Ollama
# ollama = Ollama(base_url='http://localhost:11434', model='llama3.1',messages=[{'role': 'user', 'content': 'You are a data extraction assistant.'}],
#     stream=True)
# ollama(
#     messages=[{'role': 'user', 'content': 'You are a data extraction assistant.'}],
#     stream=True
    # ) 

In [25]:
# from langchain_ollama import OllamaLLM

# llm = OllamaLLM(model="llama3.1",messages=[{'role': 'user', 'content': 'You are a data extraction assistant.'}],stream=True)
# response = llm.invoke('hii')
# print(response)

In [1]:
import ollama

llm = ollama.chat(
    model="llama3.1",
    messages=[{'role': 'user', 'content': 'You are a data extraction assistant.'}],
    stream=False
)

# for chunk in response:
#     print(chunk['message']['content'], end='', flush=True)
print(llm['message']['content'])

I can help extract specific information from various sources, including:

* Text documents (e.g., PDFs, Word files)
* Web pages
* Databases
* Spreadsheets
* Other digital content

What type of data do you need extracted? Please provide me with the following details:

1. **Source**: Where is the data located (e.g., URL, file path, database name)?
2. **Format**: What format is the data in (e.g., CSV, JSON, plain text)?
3. **Data to extract**: Specify what specific information you need extracted (e.g., names, dates, prices).
4. **Any specific requirements**: Are there any particular rules or formatting requirements for the extracted data?

Let's get started!


In [22]:
# for chunk in llm:
#     print(chunk['message']['content'], end='', flush=True)

TypeError: tuple indices must be integers or slices, not str

In [21]:
# from langchain.llms import Ollama
# llm = Ollama(base_url='http://localhost:11434',
#                 model='llama3.1')
# print(llm(prompt))

In [5]:
# try:
#     llm = ollama.chat(
#     model='llama3.1',
#     messages=[{'role': 'user', 'content': 'You are a data extraction assistant.'}],
#     stream=True
#     ) 
#     for chunk in llm:
#         print(chunk['message']['content'], end='', flush=True)
# except Exception as e:
#     None

In [1]:
!pip install easyocr

  Obtaining dependency information for easyocr from https://files.pythonhosted.org/packages/bb/84/4a2cab0e6adde6a85e7ba543862e5fc0250c51f3ac721a078a55cdcff250/easyocr-1.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/0d/4a/e51420d46cfc90562e85af2fee912237c662ab31140ab179e49bd69401d6/torch-2.5.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision>=0.5 from https://files.pythonhosted.org/packages/69/55/ce836703ff77bb21582c3098d5311f8ddde7eadc7eab04be9561961f4725/torchvision-0.20.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/26/d0/22f68eb23eea053a31655960f133c0be9726c6a881547e6e9e7e2a946c4f/opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/ea/c2/5ecadc5fcccefaece775feadcd795060adf5c3b


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import fitz  # pymupdf

def extract_table_and_images(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)

    # Extracting tables (if any) from each page
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        
        # Extract text and look for table structures
        text = page.get_text("text")
        print(text)
        
        # You can implement custom logic to recognize tables in the text here
        # For now, just print the text
        print(f"Page {page_num + 1} Text:\n{text[:500]}")  # Printing a snippet of text
        
        # Extract images
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_page_{page_num + 1}_img_{img_index + 1}.png"
            
            # Save the image
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
            print(f"Saved image: {image_filename}")

    # Close the document
    doc.close()

# Example usage
pdf_path = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data/46500152___20240912_060559-redacted.pdf'
print(extract_table_and_images(pdf_path))


In [2]:
import easyocr

reader = easyocr.Reader(['en'])  # Initialize for English language
results = reader.readtext('C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/research/image_page_1_img_1.png')  # Replace with your image path

for (bbox, text, prob) in results:
    print(f"Detected text: {text} (Confidence: {prob})")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteDetected text: ROOFING WORK ORDER (Confidence: 0.9074893646359959)
Detected text: Crew: (Confidence: 0.9998794606765605)
Detected text: Job #: (Confidence: 0.9839771342886046)
Detected text: 46500 (Confidence: 0.9973271356204866)
Detected text: Start Date: (Confidence: 0.8740533065162321)
Detected text: Supply Co: (Confidence: 0.9748557275163592)
Detected text: Crew Contact 8am (Confidence: 0.8319221732972323)
Detected text: Noon (Confidence: 0.990284264087677)
Detected text: 4pm (Confidence: 0.9947997073543151)
Detected text: Email Date:_ (Confidence: 0.7562557059771977)
Detected text: 12 (Confidence: 0.9882900409558113)
Detected text: 2} (Confidence: 0.3620489560004472)
Detected text: Delivery Communication (Confidence: 0.9997090073754178)
Detected text: Date to Deliver: (Confidence: 0.9975844441584428)
Detected text: Rep: (Confidence: 0.9998525977134705)
Detected text: sent text (Confidence: 0.915932836799

In [3]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# loading the vectorstore
vectorstore = Chroma(collection_name="example_collection",persist_directory="./chroma_db", embedding_function=OllamaEmbeddings(model="llama3"))

import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)

# loading the Llama3 model
llm = Ollama(model="llama3")

# using the vectorstore as the retriever
retriever = vectorstore.as_retriever()

# formating the docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# loading the QA chain from langchain hub
rag_prompt = hub.pull("rlm/rag-prompt")

# creating the QA chain
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# running the QA chain in a loop until the user types "exit"
while True:
    question = input("Question: ")
    if question.lower() == "exit":
        break
    answer = qa_chain.invoke(question)

    print(f"\nAnswer: {answer}\n")





I'd be happy to help with extracting data from various sources. What kind of data do you need extracted? Is it from:

1. Documents (e.g., PDFs, Word documents)?
2. Spreadsheets (e.g., Excel, Google Sheets)?
3. Websites or web pages?
4. Social media platforms?
5. Databases or APIs?
6. Something else?

Please provide more information about the data you need extracted, such as:

* The format of the data
* The source where the data is located
* Any specific fields or columns you're interested in extracting
* Any filters or conditions that apply to the extraction

I'll do my best to assist you with extracting the data efficiently!


In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# loading the vectorstore
vectorstore = Chroma(collection_name="example_collection",persist_directory="./chroma_db", embedding_function=OllamaEmbeddings(model="llama3"))

import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)

# loading the Llama3 model
llm = Ollama(model="llama3")

# using the vectorstore as the retriever
retriever = vectorstore.as_retriever()








In [ ]:
import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)


In [69]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional

# Define the Pydantic models with Field descriptions and default values set to None

class Address(BaseModel):
    name: Optional[str] = Field(default=None, description="Name associated with the address")
    address1: Optional[str] = Field(default=None, description="Primary address line")
    address2: Optional[str] = Field(default=None, description="Secondary address line")
    address3: Optional[str] = Field(default=None, description="Tertiary address line (if applicable)")
    city: Optional[str] = Field(default=None, description="City name")
    state: Optional[str] = Field(default=None, description="State or region")
    country: Optional[str] = Field(default=None, description="Country name")
    country_code: Optional[str] = Field(default=None, description="ISO country code")
    zip: Optional[str] = Field(default=None, description="Postal/ZIP code")

class Contact(BaseModel):
    name: Optional[str] = Field(default=None, description="Contact's full name")
    email: Optional[str] = Field(default=None, description="Contact's email address")
    contact_number: Optional[str] = Field(default=None, description="Contact's phone number")

class Header(BaseModel):
    ship_to: Optional[Address] = Field(default=None, description="Shipping address details")
    bill_to: Optional[Address] = Field(default=None, description="Billing address details")
    vendor: Optional[Address] = Field(default=None, description="Vendor details")
    buyer_contact: Optional[Contact] = Field(default=None, description="Contact details for the buyer")
    shipping_contact: Optional[Contact] = Field(default=None, description="Contact details for shipping")
    project_number: Optional[str] = Field(default=None, description="Unique project identifier")
    purchase_order_number: Optional[str] = Field(default=None, description="Purchase order reference")
    job_name: Optional[str] = Field(default=None, description="Name of the job associated with the order")
    job_number: Optional[str] = Field(default=None, description="Unique identifier for the job")
    quote_number: Optional[str] = Field(default=None, description="Quote reference number")
    date_ordered: Optional[str] = Field(default=None, description="Date the order was placed")
    delivery_date: Optional[str] = Field(default=None, description="Expected delivery date")
    shipping_instructions: Optional[str] = Field(default=None, description="Special shipping instructions")
    notes: Optional[str] = Field(default=None, description="Additional notes for the order")
    ship_via: Optional[str] = Field(default=None, description="Shipping method")
    payment_terms: Optional[str] = Field(default=None, description="Payment terms for the order")

class LineItem(BaseModel):
    line_no: Optional[str] = Field(default=None, description="Line item number")
    on_hand: Optional[str] = Field(default=None, description="Quantity on hand")
    to_buy: Optional[str] = Field(default=None, description="Quantity to be purchased")
    quantity: Optional[int] = Field(default=None, description="Total quantity of the item")
    uom: Optional[str] = Field(default=None, description="Unit of measure (e.g., PCS, KG)")
    unit_price: Optional[str] = Field(default=None, description="Price per unit")
    currency: Optional[str] = Field(default=None, description="Currency of the unit price")
    part_numbers: Optional[str] = Field(default=None, description="Part number for the item")
    product_description: Optional[str] = Field(default=None, description="Description of the product")
    spell_corrected_product_description: Optional[str] = Field(default=None, description="Corrected product description (if applicable)")

class ExtractionItem(BaseModel):
    header: Optional[Header] = Field(default=None, description="Metadata and general order information")
    line_items: Optional[List[LineItem]] = Field(default=None, description="List of items included in the order")

class Extraction(BaseModel):
    extraction: List[ExtractionItem] = Field(..., description="List of extracted orders")
    
# Example function to validate data
def validate_data(input_data: dict):
    try:
        # Parse and validate data
        validated_data = Extraction(**input_data)
        print("Validation successful!")
        return validated_data.dict()  
    except ValidationError as e:
        # Handle validation errors
        print("Validation failed:")
        for error in e.errors():
            print(f"Field: {error['loc']}, Error: {error['msg']}, Type: {error['type']}")
        raise e  # Optionally re-raise the exception for further handling


In [70]:
import os
import fitz  # PyMuPDF
import easyocr

def process_pdfs(input_folder, output_folder):
    # Ensure output folders for images and text exist
    image_output_folder = os.path.join(output_folder, "images")
    text_output_folder = os.path.join(output_folder, "text")
    os.makedirs(image_output_folder, exist_ok=True)
    os.makedirs(text_output_folder, exist_ok=True)
    
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])

    # Iterate through all PDF files in the input folder
    for pdf_file in os.listdir(input_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(input_folder, pdf_file)
            print(f"Processing PDF: {pdf_file}")

            # Open the PDF
            doc = fitz.open(pdf_path)

            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                
                # Create a single text file per page to combine all outputs
                text_filename = os.path.join(
                    text_output_folder, f"{os.path.splitext(pdf_file)[0]}_page_{page_num + 1}.txt"
                )
                with open(text_filename, "w", encoding="utf-8") as text_file:
                    # Extract page text
                    text = page.get_text("text")
                    text_file.write("Extracted Text from Page:\n")
                    text_file.write(text + "\n\n")
                    print(f"Saved page text for Page {page_num + 1}: {text_filename}")

                    # Extract images and perform OCR
                    images = page.get_images(full=True)
                    for img_index, img in enumerate(images):
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_bytes = base_image["image"]
                        image_filename = os.path.join(
                            image_output_folder, 
                            f"{os.path.splitext(pdf_file)[0]}_page_{page_num + 1}_img_{img_index + 1}.png"
                        )
                        with open(image_filename, "wb") as img_file:
                            img_file.write(image_bytes)
                        print(f"Saved image: {image_filename}")

                        # Perform OCR on the image
                        ocr_results = reader.readtext(image_filename)
                        text_file.write(f"OCR Results for Image {img_index + 1}:\n")
                        for (_, ocr_text, prob) in ocr_results:
                            text_file.write(f"{ocr_text} (Confidence: {prob})\n")
                        text_file.write("\n")
                        print(f"Saved OCR results for image: {image_filename}")

            # Close the document
            doc.close()

    print("Processing complete.")

# Example usage
input_folder = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/data'
output_folder = 'C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output'
process_pdfs(input_folder, output_folder)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Processing PDF: 46500152___20240912_060559-redacted.pdf
Saved page text for Page 1: C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output\text\46500152___20240912_060559-redacted_page_1.txt
Saved image: C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output\images\46500152___20240912_060559-redacted_page_1_img_1.png
Saved OCR results for image: C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output\images\46500152___20240912_060559-redacted_page_1_img_1.png
Processing PDF: 99907349 Roof 3-redacted.pdf
Saved page text for Page 1: C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output\text\99907349 Roof 3-redacted_page_1.txt
Saved image: C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output\images\99907349 Roof 3-redacted_page_1_img_1.png
Saved OCR results for image: C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output\images\99907349 Roof 3-redacted_pag

In [12]:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


loader = DirectoryLoader("../output/text/", glob = "./*.txt", loader_cls= TextLoader)

document = loader.load()

In [13]:

document

[Document(metadata={'source': '..\\output\\text\\46500152___20240912_060559-redacted_page_1.txt'}, page_content='Extracted Text from Page:\n\n\nOCR Results for Image 1:\nROOFING WORK ORDER (Confidence: 0.9074893646359959)\nCrew: (Confidence: 0.9998794606765605)\nJob #: (Confidence: 0.9839771342886046)\n46500 (Confidence: 0.9973271356204866)\nStart Date: (Confidence: 0.8740533065162321)\nSupply Co: (Confidence: 0.9748557275163592)\nCrew Contact 8am (Confidence: 0.8319221732972323)\nNoon (Confidence: 0.990284264087677)\n4pm (Confidence: 0.9947997073543151)\nEmail Date:_ (Confidence: 0.7562557059771977)\n12 (Confidence: 0.9882900409558113)\n2} (Confidence: 0.3620489560004472)\nDelivery Communication (Confidence: 0.9997090073754178)\nDate to Deliver: (Confidence: 0.9975844441584428)\nRep: (Confidence: 0.9998525977134705)\nsent text (Confidence: 0.915932836799662)\nsent email (Confidence: 0.9634564005781879)\nAM (Confidence: 0.9994721936755578)\nPM (Confidence: 0.9994458982098452)\nPick Up 

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


loader = DirectoryLoader("../output/text/", glob = "./*.txt", loader_cls= TextLoader)

document = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
text = text_splitter.split_documents(document)

from langchain_community import embeddings

persist_directory = 'db'

# 2. Convert documents to Embeddings and store them
vectorstore = Chroma.from_documents(
    documents=text,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
)
# vectorstore.persist()
# vectorstore = None
# retriever = vectorstore.as_retriever()

# # Now we can load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory=persist_directory,
#                   embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'))

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [15]:
text

[Document(metadata={'source': '..\\output\\text\\46500152___20240912_060559-redacted_page_1.txt'}, page_content='Extracted Text from Page:'),
 Document(metadata={'source': '..\\output\\text\\46500152___20240912_060559-redacted_page_1.txt'}, page_content='OCR Results for Image 1:\nROOFING WORK ORDER (Confidence: 0.9074893646359959)'),
 Document(metadata={'source': '..\\output\\text\\46500152___20240912_060559-redacted_page_1.txt'}, page_content='Crew: (Confidence: 0.9998794606765605)\nJob #: (Confidence: 0.9839771342886046)'),
 Document(metadata={'source': '..\\output\\text\\46500152___20240912_060559-redacted_page_1.txt'}, page_content='46500 (Confidence: 0.9973271356204866)\nStart Date: (Confidence: 0.8740533065162321)'),
 Document(metadata={'source': '..\\output\\text\\46500152___20240912_060559-redacted_page_1.txt'}, page_content='Supply Co: (Confidence: 0.9748557275163592)\nCrew Contact 8am (Confidence: 0.8319221732972323)'),
 Document(metadata={'source': '..\\output\\text\\4650015

In [ ]:
!ollama pull nomic-embed-text
!ollama pull llama3.2:1b

In [16]:
from langchain_community import embeddings

persist_directory = 'db'

# 2. Convert documents to Embeddings and store them
vectorstore = Chroma.from_documents(
    documents=text,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
)
# vectorstore.persist()
# vectorstore = None
# retriever = vectorstore.as_retriever()

# # Now we can load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory=persist_directory,
#                   embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'))

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

C:\Users\kisha\AppData\Local\Temp\ipykernel_8280\3816354697.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),


In [51]:
from typing import List, Optional

from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

import pandas as pd
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain


from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [71]:
# from pydantic import ValidationError
# from langchain.schema import from_pydantic

# Define the extraction validator using `from_pydantic`
schema, extraction_validator = from_pydantic(
    Extraction,
    description="Extract structured data including headers and line items from order-related documents.",
    examples=[
        (
            """
            Order: PO12345
            Ship to: John Doe, 123 Main St, Anytown, USA
            Line Item 1: Part #ABC123, Qty: 10, Price: $100
            """,
            {
                "extraction": [
                    {
                        "header": {
                            "purchase_order_number": "PO12345",
                            "ship_to": {
                                "name": "John Doe",
                                "address1": "123 Main St",
                                "city": "Anytown",
                                "country": "USA",
                            },
                        },
                        "line_items": [
                            {
                                "line_no": "1",
                                "part_numbers": "ABC123",
                                "quantity": 10,
                                "unit_price": "$100",
                            }
                        ],
                    }
                ]
            },
        )
    ],
    many=False  # Adjust based on whether you want a single or multiple extraction items
)

# Validate an example input
# try:
#     example_input = {
#         "extraction": [
#             {
#                 "header": {
#                     "purchase_order_number": "PO12345",
#                     "ship_to": {
#                         "name": "John Doe",
#                         "address1": "123 Main St",
#                         "city": "Anytown",
#                         "country": "USA",
#                     },
#                 },
#                 "line_items": [
#                     {
#                         "line_no": "1",
#                         "part_numbers": "ABC123",
#                         "quantity": 10,
#                         "unit_price": "$100",
#                     }
#                 ],
#             }
#         ]
#     }
#     validated_output = validate_data(example_input)
#     print("Validated Output:", validated_output)
# except ValidationError as e:
#     print("Validation Error:", e)


Validation successful!
Validated Output: {'extraction': [{'header': {'ship_to': {'name': 'John Doe', 'address1': '123 Main St', 'address2': None, 'address3': None, 'city': 'Anytown', 'state': None, 'country': 'USA', 'country_code': None, 'zip': None}, 'bill_to': None, 'vendor': None, 'buyer_contact': None, 'shipping_contact': None, 'project_number': None, 'purchase_order_number': 'PO12345', 'job_name': None, 'job_number': None, 'quote_number': None, 'date_ordered': None, 'delivery_date': None, 'shipping_instructions': None, 'notes': None, 'ship_via': None, 'payment_terms': None}, 'line_items': [{'line_no': '1', 'on_hand': None, 'to_buy': None, 'quantity': 10, 'uom': None, 'unit_price': '$100', 'currency': None, 'part_numbers': 'ABC123', 'product_description': None, 'spell_corrected_product_description': None}]}]}


In [53]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="ollama run llama3.2:1b")

In [74]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json",validator=extraction_validator,input_formatter="triple_quotes")

In [75]:
document_extraction_results = await extract_from_documents(
        chain, text, max_concurrency=5, use_uid=False, return_exceptions=True
    )

In [76]:
document_extraction_results

[2 validation errors for GenerateRequest
 format.literal['json']
   Input should be 'json' [type=literal_error, input_value='', input_type=str]
     For further information visit https://errors.pydantic.dev/2.9/v/literal_error
 format.dict[str,any]
   Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
     For further information visit https://errors.pydantic.dev/2.9/v/dict_type,
 2 validation errors for GenerateRequest
 format.literal['json']
   Input should be 'json' [type=literal_error, input_value='', input_type=str]
     For further information visit https://errors.pydantic.dev/2.9/v/literal_error
 format.dict[str,any]
   Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
     For further information visit https://errors.pydantic.dev/2.9/v/dict_type,
 2 validation errors for GenerateRequest
 format.literal['json']
   Input should be 'json' [type=literal_error, input_value='', input_type=str]
     For further informa

In [80]:
from kor import from_pydantic
from typing import List, Optional
from pydantic import BaseModel, Field
class Person(BaseModel):
    first_name: str = Field(description="The first name of a person")
schema, validator = from_pydantic(
    Person,
    description="Personal Information",  # <-- Description
    examples=[  # <-- Object level examples
        ("Alice and Bob are friends", [{"first_name": "Alice"}, {"first_name": "Bob"}])
    ],
    many=True,  # <-- Note Many = True
)

chain = create_extraction_chain(llm, schema, validator=validator)


In [90]:
from pydantic import BaseModel

class TextInput(BaseModel):
    text: str
    
input_data = TextInput(text="My name is Bobby. My brother's name Joe.")

chain.invoke(input_data)

ValidationError: 3 validation errors for HumanMessage
content.str
  Input should be a valid string [type=string_type, input_value=TextInput(text="My name i...My brother's name Joe."), input_type=TextInput]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=('text', "My name is Bobb...My brother's name Joe."), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=('text', "My name is Bobb...My brother's name Joe."), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [61]:
input_text = """
This store sells various products such as a Red Chair for $45.99, a Blue Table for $89.99, 
and a Green Lamp for $22.50. All items come with a one-year warranty.
"""

output = chain.invoke(input_text)
print(output)

ValidationError: 2 validation errors for GenerateRequest
format.literal['json']
  Input should be 'json' [type=literal_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
format.dict[str,any]
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [57]:
response = chain.invoke(text)

ValidationError: 233 validation errors for HumanMessage
content.str
  Input should be a valid string [type=string_type, input_value=[Document(metadata={'sour... Results for Image 1:')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=Document(metadata={'sourc...racted Text from Page:'), input_type=Document]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=Document(metadata={'sourc...racted Text from Page:'), input_type=Document]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type
content.list[union[str,dict[any,any]]].1.str
  Input should be a valid string [type=string_type, input_value=Document(metadata={'sourc...e: 0.9074893646359959)'), input_type=Docum

In [29]:
result = chain.invoke()

TypeError: RunnableSequence.invoke() missing 1 required positional argument: 'input'

In [63]:
!pip -q install kor markdownify


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
chain = create_extraction_chain(
    llm,
    schema,
    encoder_or_encoder_class="json",
    validator=extraction_validator,
    input_formatter="triple_quotes",
)

In [1]:
chain = create_extraction_chain(llm, schema)

NameError: name 'create_extraction_chain' is not defined

In [68]:
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [71]:
formatted_prompt = chain.steps[0].prompt.format(text="[user input]")
print(formatted_prompt)

AttributeError: 'ExtractionPromptTemplate' object has no attribute 'prompt'

In [66]:
print(chain.prompt.format_prompt(text="[user input]").to_string())

AttributeError: 'RunnableSequence' object has no attribute 'prompt'

In [67]:
with get_openai_callback() as cb:
    document_extraction_results = await extract_from_documents(
        chain, text, max_concurrency=5, use_uid=False, return_exceptions=True
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [ ]:
import json

def extract_restaurant_info(json_data):
    for record in json_data:
        restaurant_list = record.get('data', {}).get('restaurant', [])
        for restaurant in restaurant_list:
            name = restaurant.get('name', '')
            location = restaurant.get('location', '')
            style = restaurant.get('style', '')
            top_dish = restaurant.get('top_dish', '')
            
            # If style is not specified, we'll just say "Cuisine not specified"
            style = style if style else 'Cuisine not specified'

            print(f'Restaurant Name: {name}\nLocation: {location}\nStyle: {style}\nTop Dish: {top_dish}\n')

In [ ]:
# docs = retriever.get_relevant_documents("How much money did Microsoft raise?")


In [ ]:
# persist_directory = 'db'

# # embedding = OllamaEmbeddings()

# vectordb = Chroma.from_documents(documents=text,
#                                  embedding=embedding,
#                                  persist_directory=persist_directory)


# # persiste the db to disk
# vectordb.persist()
# vectordb = None
     

# # Now we can load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory=persist_directory,
#                   embedding_function=embedding)

In [ ]:
# from langchain.chains import RetrievalQA
     

# # llm=OpenAI()

# # create the chain to answer questions
# qa_chain = RetrievalQA.from_chain_type(llm=llm,
#                                   chain_type="stuff",
#                                   retriever=retriever,
#                                   return_source_documents=True)
     

# ## Cite sources
# def process_llm_response(llm_response):
#     print(llm_response['result'])
#     print('\n\nSources:')
#     for source in llm_response["source_documents"]:
#         print(source.metadata['source'])
     

# # break it down
# query = "What is the news about Pando?"
# llm_response = qa_chain(query)
# process_llm_response(llm_response)

In [13]:
!pip install langchain-ollama


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="ollama run llama3.2:1b")


In [24]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 74701a8c35f6... 100% ▕████████████████▏ 1.3 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [34]:
!set OLLAMA_HOST=127.0.0.1:11435

In [36]:
!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: Only one usage of each socket address (protocol/network address/port) is normally permitted.


In [42]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="ollama run llama3.2:1b"
)

In [43]:
from langchain import embeddings
persist_directory = 'db'

# embedding = OllamaEmbeddings()

vectordb = Chroma.from_documents(documents=text,
                                 embedding=embedding,
                                 persist_directory=persist_directory)


# persiste the db to disk
vectordb.persist()
vectordb = None
     

# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)
     

ResponseError: model "ollama run llama3.2:1b" not found, try pulling it first

In [ ]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")
len(docs)
docs


retriever = vectordb.as_retriever(search_kwargs={"k": 2})



In [ ]:
from langchain.chains import RetrievalQA
     

llm=OpenAI()

# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)
     

## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])
     

# break it down
query = "What is the news about Pando?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [11]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [44]:
!ollama pull nomic-embed-text
!ollama pull llama3.2:1b

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90

In [24]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 74701a8c35f6... 100% ▕████████████████▏ 1.3 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [18]:
!pip install tiktoken


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter

model_local = ChatOllama(model="llama")

# 1. Split data into chunks
urls = [
    "https://ollama.com/",
    "https://ollama.com/blog/windows-preview",
    "https://ollama.com/blog/openai-compatibility",
]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs_list)

# 2. Convert documents to Embeddings and store them
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

# 3. Before RAG
print("Before RAG\n")
before_rag_template = "What is {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "Ollama"}))

# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("What is Ollama?"))

# loader = PyPDFLoader("Ollama.pdf")
# doc_splits = loader.load_and_split()

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\kisha\AppData\Local\Temp\ipykernel_13120\3284211468.py:12: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model_local = ChatOllama(model="mistral")


ImportError: Could not import tiktoken python package. This is needed in order to calculate max_tokens_for_prompt. Please install it with `pip install tiktoken`.

In [33]:
from pydantic import BaseModel, Field
from typing import List

class DocumentData(BaseModel):
    title: str = Field(description="Title of the document")
    content: str = Field(description="Main content of the document")
    keywords: List[str] = Field(default=[], description="Keywords related to the document")

In [38]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI

model = OllamaLLM(model='llama')

# Define a query to generate a response based on the document data
query = "Summarize the main points of the document."

parser = PydanticOutputParser(pydantic_object=DocumentData)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Combine the prompt with the model and parser
chain = prompt | model | parser

In [40]:
with open("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt", 'r') as f:
    conversation = f.read()

In [41]:
# pdf_text = extract_text_from_text("C:/Users/kisha/OneDrive/Desktop/revalgo assignment/revalgo-assignment/output/text/46500152___20240912_060559-redacted_page_1.txt")
response = chain.invoke({"query": query})

ValidationError: 2 validation errors for GenerateRequest
format.literal['json']
  Input should be 'json' [type=literal_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
format.dict[str,any]
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [ ]:
document_data = DocumentData(**response)
print(document_data)

In [19]:
from typing import List, Optional
import itertools
import requests

import pandas as pd
from pydantic import BaseModel, Field, field_validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain
from kor.documents.html import MarkdownifyHTMLProcessor
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_openai import ChatOpenAI


In [7]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="ollama run llama3.2:1b")



In [20]:
class ShowOrMovie(BaseModel):
    name: str = Field(
        description="The name of the movie or tv show",
    )
    season: Optional[str] = Field(
        description="Season of TV show. Extract as a digit stripping Season prefix.",
    )
    year: Optional[str] = Field(
        description="Year when the movie / tv show was released",
    )
    latest_episode: Optional[str] = Field(
        description="Date when the latest episode was released",
    )
    link: Optional[str] = Field(description="Link to the movie / tv show.")

    # rating -- not included because rating on rottentomatoes is in the html elements
    # you could try extracting it by using the raw HTML (rather than markdown)
    # or you could try doing something similar on imdb

    @field_validator("name")
    def name_must_not_be_empty(cls, v):
        if not v:
            raise ValueError("Name must not be empty")
        return v


In [21]:
schema, extraction_validator = from_pydantic(
    ShowOrMovie,
    description="Extract information about popular movies/tv shows including their name, year, link and rating.",
    examples=[
        (
            "[Rain Dogs Latest Episode: Apr 03](/tv/rain_dogs)",
            {"name": "Rain Dogs", "latest_episode": "Apr 03", "link": "/tv/rain_dogs"},
        )
    ],
    many=True,
)

In [22]:
chain = create_extraction_chain(
    llm,
    schema,
    encoder_or_encoder_class="csv",
    validator=extraction_validator,
    # input_formatter="triple_quotes",
)

In [ ]:
document_extraction_results = await extract_from_documents(
        chain, split_docs, max_concurrency=5, use_uid=False, return_exceptions=True
    )

In [24]:
document_extraction_results

[[{'uid': '0',
   'source_uid': '0',
   'data': {},
   'raw': '{\n    "customerInformation": {\n        "Derek": {\n            "name": "Derek",\n            "address": "Crown Point Rd, Man, (701) 390"\n        },\n        "Man": {\n            "name": "Man",\n            "address": "(701) 390"\n        }\n    },\n    "crewInformation": {\n        "Mefcl": {\n            "phone": "2lss",\n            "email": "Seles-2lss"\n        },\n        "Seles-2lss": {\n            "phone": "2lss",\n            "email": "Seles-2lss"\n        }\n    },\n    "productionInformation": {\n        "Crystal": {\n            "phone": "Mefcl",\n            "email": "Seles-2lss"\n        },\n        "Build Communication": {\n            "phone": "Mefcl",\n            "email": "Seles-2lss"\n        }\n    },\n    "deliveryInstructions": {\n        "placeMaterialsOnTheLeftSideOfTheDrivewayInTheFrontOfTheGarage": [\n            "Place materials on the LEFT SIDE OF THE DRIVEWAY, IN FRONT OF THE GARAGE."\n     

In [19]:
validated_data = list(
    itertools.chain.from_iterable(
        extraction["validated_data"] for extraction in document_extraction_results
    )
)

TypeError: 'pydantic_core._pydantic_core.ValidationError' object is not subscriptable

In [25]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2:1b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that shorter wavelengths of light, like blue and violet, are scattered more than longer wavelengths, like red and orange, by the tiny molecules of gases in the Earth's atmosphere.

Here's what happens:

1. When sunlight enters the Earth's atmosphere, it consists of a spectrum of colors, including all the different wavelengths.
2. The shorter wavelengths of light, such as blue and violet, are scattered more than the longer wavelengths, like red and orange, by the gas molecules.
3. This scattering effect is more pronounced because the smaller molecules in the atmosphere scatter the light in all directions.
4. As a result, the blue light is distributed throughout the sky, giving it a blue appearance.

This process is more noticeable during the daytime when the sun is high in the sky. At sunrise and sunset, the light has to travel through more of the

In [26]:
!ollama pull nomic-embed-text
!ollama pull llama3.2:1b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pu

In [29]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.2:1b")

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Question: {question}")


model = OllamaLLM(model="llama3.2:1b")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

ValidationError: 2 validation errors for GenerateRequest
format.literal['json']
  Input should be 'json' [type=literal_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
format.dict[str,any]
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [39]:
chain.invoke("What is LangChain?")


ValidationError: 2 validation errors for GenerateRequest
format.literal['json']
  Input should be 'json' [type=literal_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
format.dict[str,any]
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [45]:
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM

# Define prompt template
prompt = PromptTemplate.from_template("Answer the following question")

# Initialize model
model = OllamaLLM(model="llama3.2:1b")

# Create the chain
chain = prompt | model

# Pass a dictionary input (as required by the template)
chain.invoke("What is LangChain?")

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT 

In [65]:
from langchain_ollama import OllamaLLM

# Initialize the model
model = OllamaLLM(model="llama3.2:1b",temperature=0)  # Ensure the model name matches your setup
print(model)

# Pass input as a dictionary with a 'prompt' key
# print(response)


OllamaLLM
Params: {}


In [66]:
print(model.input_schema.schema())


{'$defs': {'AIMessage': {'additionalProperties': True, 'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.', 'properties': {'content': {'anyOf': [{'type': 'string'}, {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}, 'type': 'array'}], 'title': 'Content'}, 'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'}, 'response_metadata': {'title': 'Response Metadata', 'type': 'object'}, 'type': {'const': 'ai', 'default': 'ai', 'enum': ['ai'], 'title': 'Type', 'type': 'string'}, 'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Name'}, 'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Id'}, 'example': {'default': False, 'title': 'Example'

In [67]:
response = model.invoke("What is LangChain?")


ValidationError: 2 validation errors for GenerateRequest
format.literal['json']
  Input should be 'json' [type=literal_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
format.dict[str,any]
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [63]:
!pip install langchain_ollama==0.2.2

  Obtaining dependency information for langchain_ollama==0.2.2 from https://files.pythonhosted.org/packages/7f/77/219fb2290c832e33af2731246ea3328bade50756288c1e97ae73c4ccc197/langchain_ollama-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.4.0,>=0.3.27 from https://files.pythonhosted.org/packages/9a/bf/07e63d4b4c41aa49daf4a4499e8010928ce8545469f0265544f925c95fff/langchain_core-0.3.28-py3-none-any.whl.metadata
  Obtaining dependency information for ollama<1,>=0.4.4 from https://files.pythonhosted.org/packages/d3/01/815774a30d0047d464add27e2824a5b1d02ea4ad021c83590cd753e5f511/ollama-0.4.4-py3-none-any.whl.metadata
  Using cached ollama-0.4.4-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/411.6 kB ? eta -:--:--
   -- ------------------------------------- 30.7/411.6 kB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 245.8/411.6 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 411


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="llama3.2:1b")
model.invoke("Come up with 10 names for a song about parrots")

ValidationError: 2 validation errors for GenerateRequest
format.literal['json']
  Input should be 'json' [type=literal_error, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
format.dict[str,any]
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [70]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='llama3.2:1b')

In [73]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model='llama3.2:1b')

# Without bind.
chain = (
    llm
    | StrOutputParser()
)

chain.invoke("Repeat quoted words exactly: 'One two three four five.'")
# Output is 'One two three four five.'

# # With bind.
# chain = (
#     llm.bind(stop=["three"])
#     | StrOutputParser()
# )

# chain.invoke("Repeat quoted words exactly: 'One two three four five.'")
# # Output is 'One two'

"'One two three four five.'"